In [1]:
!pip install pyngrok

In [2]:
from flask import Flask, request, jsonify
import tensorflow as tf
from PIL import Image
import numpy as np
import requests
from io import BytesIO
from pyngrok import ngrok

In [3]:
port_no = 5000

In [4]:
app = Flask(__name__)
model = tf.keras.models.load_model('/content/drive/Othercomputers/My Laptop/Kuliah/Semester_6/ProyekPerangkatLunak/deployment/beef_pork_horse_classifier.h5')

ngrok.set_auth_token("2fS5nDZnBgMJIj3lOk8RTELAZnO_5zgiBpF7hX6nHo7xcjrb9")
public_url =  ngrok.connect(port_no).public_url

In [5]:
@app.route('/', methods=['GET'])
def hello_world():
    return jsonify({'message': 'Hello, World!'})

@app.route('/models', methods=['GET'])
def get_models():
    return jsonify({'models': 'MobileNetV3Large', 'framework': 'TensorFlow', 'task': 'Image Classification for Beef, Pork, and Horse', 'accuracy': '97.43%', 'input': 'URL', 'output': 'Predicted class and probabilities', 'model_url': 'https://www.kaggle.com/code/hafidhmuhammadakbar/mobilenetv3large-fix'})


@app.route('/models', methods=['POST'])
def predict():
    # Check if the request contains a URL
    if 'url' not in request.form:
        return jsonify({'error': 'No URL provided'})

    url = request.form['url']

    # Download the image from the URL
    try:
        response = requests.get(url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
    except Exception as e:
        return jsonify({'error': f'Failed to download image from URL: {str(e)}'})

    # Resize image
    image = image.resize((224, 224))

    # Preprocess image
    image = np.expand_dims(image, axis=0)

    # Make prediction
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions, axis=1)[0]
    probabilities = tf.reduce_max(predictions, axis=1) * 100

    class_names = ['Horse', 'Meat', 'Pork']
    predicted_class = class_names[predicted_label]
    probabilities_class = '%.2f' % probabilities.numpy()[0]

    return jsonify({'predicted_class': predicted_class, 'probabilities': probabilities_class})

print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)

To acces the Gloable link please click https://45d1-35-196-197-21.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:34] "GET /models HTTP/1.1" 200 -


1/1 [==============================] - 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:46] "POST /models HTTP/1.1" 200 -


1/1 [==============================] - 0s 51ms/step


INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:51] "POST /models HTTP/1.1" 200 -


1/1 [==============================] - 0s 61ms/step


INFO:werkzeug:127.0.0.1 - - [22/Apr/2024 10:44:55] "POST /models HTTP/1.1" 200 -
